# Tensorflow gpt2 implemenation

Here we define a tensorflow implementation of gpt2 as provided by https://github.com/ShenakhtPajouh/gpt2-keras
and load the weights. The goal is to duplicate as closely as possible the Pytorch implemenation
from LLMFS. See load_gpt.ipynb in ch05

In [1]:
import os
import copy
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
import tensorflow as tf
!pip install -q tiktoken
import tiktoken
import numpy as np
np.set_printoptions(precision=5)
from utils import clean_up
from utils import load_gpt2_params_from_tf_ckpt
from utils import print_layer_structure
from utils import tldr
import gc
from utils import generate_text_simple
from utils import text_to_token_ids
from utils import token_ids_to_text

In [2]:

# cleanup_vars = ['AttentionLayer', 'Block', 'Embedding', 'LayerNormalization', 'MultiLayerPerceptron', 'SelfAttention', 'Transformer', 'GPT2'
#                 , 'attn', 'block', 'blocks', 'final_norm_layer', 'k_b', 'k_w', 'layer_norm', 'layer_norm0', 'layer_norm11', 'q_w', 'v_w', 'k_w', 'xxxxx', 'gpt2', 'params', 'y']
# clean_up(cleanup_vars)
# %whos

## gpt2.py

In [35]:
# From https://github.com/ShenakhtPajouh/gpt2-keras
import tensorflow as tf
import numpy as np


def get_tensor_shape(x):
    x = tf.convert_to_tensor(x)
    static_shape = x.shape.as_list()
    if tf.executing_eagerly():
        return static_shape
    dynamic_shape = tf.shape(x)
    if static_shape is None:
        return dynamic_shape
    dynamic_shape = tf.unstack(dynamic_shape)
    shape = []
    for st, dyn in zip(static_shape, dynamic_shape):
        if st is None:
            shape.append(dyn)
        else:
            shape.append(st)
    return shape


def gelu(x):
    return 0.5*x*(1+tf.tanh(np.sqrt(2/np.pi)*(x+0.044715*tf.pow(x, 3))))

def dropout_fn(x, dropout):
    if dropout is None or dropout == 0.0:
        return x
    else:
        return tf.nn.dropout(x, rate=dropout)


class LayerNormalization(tf.keras.layers.Layer):

    def __init__(self, trainable=True, name=None):
        super().__init__(name=name, trainable=trainable)
        self.beta = None
        self.gamma = None

    def build(self, input_shape):
        self.beta = self.add_weight(name="beta", shape=input_shape[-1:], initializer=tf.zeros_initializer())
        self.gamma = self.add_weight(name="gamma", shape=input_shape[-1:], initializer=tf.ones_initializer())
        super().build(input_shape)

    def call(self, inputs, axis=-1, epsilon=1e-5):
        # mean, variance = tf.nn.moments(inputs, axis, keep_dims=True)
        mean, variance = tf.nn.moments(inputs, axis, keepdims=True)
        rdev = tf.math.rsqrt(variance + epsilon)
        x = (inputs - mean) * rdev
        output = x * self.gamma + self.beta
        return output

    def __call__(self, inputs, axis=-1, epsilon=1e-5):
        return super().__call__(inputs=inputs,
                                axis=axis, epsilon=epsilon)


class SelfAttention(tf.keras.layers.Layer):

    def __init__(self, num_attention_heads=1, size_per_head=512,
                 one_sided=True,
                 query_act=None,
                 initializer_range=0.02,
                 value_act=None,
                 key_act=None,
                 trainable=True,
                 name=None):
        super().__init__(name=name, trainable=trainable)
        # `query_layer` = [B*F, N*H]
        self.attention_size = num_attention_heads * size_per_head
        self.query_layer = tf.keras.layers.Dense(
            num_attention_heads * size_per_head,
            activation=query_act,
            name="query",
            kernel_initializer=tf.random_normal_initializer(stddev=initializer_range)
        )
        # `key_layer` = [B*T, N*H]
        self.key_layer = tf.keras.layers.Dense(
            num_attention_heads * size_per_head,
            activation=key_act,
            name="key",
            kernel_initializer=tf.random_normal_initializer(stddev=initializer_range)
        )
        # `value_layer` = [B*T, N*H]
        self.value_layer = tf.keras.layers.Dense(
            num_attention_heads * size_per_head,
            activation=value_act,
            name="value",
            kernel_initializer=tf.random_normal_initializer(stddev=initializer_range)
        )
        self.size_per_head = size_per_head
        self.num_attention_heads = num_attention_heads
        self.one_sided = one_sided

    def reshape(self, x, use_2d=False, shape=None):
        if use_2d:
            batch_size, seq_length = shape[0], shape[1]
        else:
            _shape = get_tensor_shape(x)
            batch_size, seq_length = _shape[0], _shape[1]
        x = tf.reshape(x, [batch_size, seq_length, self.num_attention_heads, self.size_per_head])
        x = tf.transpose(x, [0, 2, 1, 3])
        return x

    def final_shape(self, x, use_2d=False):
        shape = get_tensor_shape(x)
        batch_size, seq_length = shape[0], shape[2]
        x = tf.transpose(x, [0, 2, 1, 3])
        if use_2d:
            x = tf.reshape(x, [batch_size * seq_length, self.num_attention_heads * self.size_per_head])
        else:
            x = tf.reshape(x, [batch_size, seq_length, self.num_attention_heads * self.size_per_head])
        return x

    def get_mask(self, inputs_shape, cache_length=None, mask=None):
        batch_size, seq_length = inputs_shape[0], inputs_shape[2]
        if self.one_sided:
            rng = tf.range(seq_length)
            one_sided_mask = tf.less_equal(rng, tf.expand_dims(rng, 1))
            if cache_length is not None:
                prev_mask = tf.ones([seq_length, cache_length], tf.bool)
                one_sided_mask = tf.concat([prev_mask, one_sided_mask], 1)
        if mask is not None:
            if cache_length is not None:
                prev_mask = tf.ones([batch_size, cache_length], tf.bool)
                mask = tf.concat([prev_mask, mask], 1)
            if cache_length is None:
                cache_length = 0
            mask = tf.reshape(mask, [batch_size, 1, 1, seq_length + cache_length])
        if self.one_sided:
            if mask is not None:
                one_sided_mask = tf.logical_and(mask, one_sided_mask)
            return one_sided_mask
        else:
            return mask

    def attend(self, query, key, value, mask=None, dropout=None):
        dim = tf.cast(self.size_per_head, query.dtype)
        _sqrt = tf.math.sqrt(dim)
        _sqrt = tf.cast(_sqrt, query.dtype)
        coefficients = tf.matmul(query, key, transpose_b=True) / _sqrt
        # print("SelfAttention: reshape coefficients.shape=", coefficients.shape, coefficients[0][0][0][:3])
        if mask is not None:
            mask = tf.cast(mask, coefficients.dtype)
            coefficients = coefficients * mask - (1 - mask) * 1e5
        coefficients = tf.math.softmax(coefficients, -1)
        coefficients = dropout_fn(coefficients, dropout)
        results = tf.matmul(coefficients, value)
        # print("SelfAttention: results coefficients.shape=", results.shape, results[0][0][0][:3])
        return results

    def call(self, inputs, cache=None, mask=None,
             attention_dropout=None, return_cache=False,
             use_2d=False, shape=None):
        """
        inputs: a tensor of shape [batch_size, seq_length, dim] if use_2d is false,
                else a tensor of shape [batch_size * seq_length, dim]
        cache: A dictionary consist of key and value from previous calls.
        mask: a boolean tensor of shape [batch_size, seq_length]
        attention_probs_dropout_prob: dropout use for attention mechanism
        return_cache: if True, it returns key and values as besides layer output
        use_2d: if it is True, the model uses 2D matrices as inputs and outputs
        shape: if use_2d is True, then the shape is [batch_size, seq_length]
        """
        print(f".... input to    SelfAttention: {tldr(inputs)}")
        query = self.query_layer(inputs)
        # print("SelfAttention: query.shape=", query.shape, query[0][0][:3])
        key = self.key_layer(inputs)
        value = self.value_layer(inputs)
        if use_2d and shape is None:
            raise ValueError("if use_2d is True, then the shape must be specified")
        query = self.reshape(query, use_2d, shape)
        # print("SelfAttention: reshape query.shape=", query.shape, query[0][0][0][:3])
        key = self.reshape(key, use_2d, shape)
        value = self.reshape(value, use_2d, shape)
        cache_length = None
        if cache is not None:
            key = tf.concat([cache["key"], key], 2)
            value = tf.concat([cache["value"], value], 2)
            cache_length = get_tensor_shape(cache["key"])[2]
        inputs_shape = get_tensor_shape(query)
        mask = self.get_mask(inputs_shape, cache_length, mask)
        result = self.attend(query, key, value, mask, attention_dropout)
        result = self.final_shape(result, use_2d)
        print(f".... output from SelfAttention: {tldr(result)}")
        if return_cache:
            cache = {"key": key, "value": value}
            return result, cache
        else:
            return result

    def __call__(self, inputs, cache=None, mask=None,
             attention_dropout=None, return_cache=False,
             use_2d=False, shape=None):
        """
        inputs: a tensor of shape [batch_size, seq_length, dim] if use_2d is false,
                else a tensor of shape [batch_size * seq_length, dim]
        cache: A dictionary consist of key and value from previous calls.
        mask: a boolean tensor of shape [batch_size, seq_length]
        attention_probs_dropout_prob: dropout use for attention mechanism
        return_cache: if True, it returns key and values as besides layer output
        use_2d: if it is True, the model uses 2D matrices as inputs and outputs
        shape: if use_2d is True, then the shape is [batch_size, seq_length]
        """
        return super().__call__(
            inputs=inputs,
            cache=cache,
            mask=mask,
            attention_dropout=attention_dropout,
            return_cache=return_cache,
            use_2d=use_2d,
            shape=shape
        )

class AttentionLayer(tf.keras.layers.Layer):

    def __init__(self, config, name=None, trainable=True, initializer_range=0.02):
        super().__init__(name=name, trainable=trainable)
        self.layer_norm = LayerNormalization(name="layer_norm")
        self.self_attention = SelfAttention(num_attention_heads=config["n_head"],
                                            size_per_head=config["n_embd"] // config["n_head"],
                                            initializer_range=initializer_range,
                                            name="self"
                                            )
        self.projection = tf.keras.layers.Dense(units=config["n_embd"],
                                                kernel_initializer=tf.random_normal_initializer(stddev=initializer_range),
                                                name="projection")


    def call(self, inputs, cache=None, dropout=None, attention_dropout=None,
             return_cache=False, use_2d=False, shape=None):
        """

        inputs: a tensor of shape [batch_size, seq_length, dim] if use_2d is False, else [batch_size * seq_length, dim]
        cache: (Optional): a dictionary of tensors key and value from previous calls.
        return_cache: if True, returns a dictionary of key and value tensors besides layer output.
        use_2d: if is True then the inputs and outputs are 2D tensors instead of 3D (for tpu performance)
        shape: if use_2d then it's [batch_size, seq_length]
        """
        print(f".... input to    layer_norm: {tldr(inputs)}")
        x = self.layer_norm(inputs)
        print(f".... output from layer_norm: {tldr(x)}")
        x = self.self_attention(x, attention_dropout=attention_dropout,
                                cache=cache,
                                return_cache=return_cache,
                                use_2d=use_2d,
                                shape=shape)
        if return_cache:
            x, cache = x
        print(f".... input to    projection: {tldr(x)}")            
        x = self.projection(x)
        print(f".... output from projection: {tldr(x)}")        
        # x = dropout_fn(x, dropout)
        if return_cache:
            return x, cache
        else:
            return x

    def __call__(self, inputs, cache=None, dropout=None, attention_dropout=None,
                 return_cache=False, use_2d=False, shape=None):
        """

        inputs: a tensor of shape [batch_size, seq_length, dim] if use_2d is False, else [batch_size * seq_length, dim]
        cache: (Optional): a dictionary of tensors key and value from previous calls.
        return_cache: if True, returns a dictionary of key and value tensors besides layer output.
        use_2d: if is True then the inputs and outputs are 2D tensors instead of 3D (for tpu performance)
        shape: if use_2d then it's [batch_size, seq_length]
        """
        return super().__call__(
            inputs=inputs,
            cache=cache,
            dropout=dropout,
            attention_dropout=attention_dropout,
            return_cache=return_cache,
            use_2d=use_2d,
            shape=shape
        )



class MultiLayerPerceptron(tf.keras.layers.Layer):

    def __init__(self, activation_fn=None, embedding_size=768,
                 perceptron_size=3072, trainable=True,
                 initializer_range=0.02, name=None):
        super().__init__(name=name, trainable=trainable)
        self.layer_norm = LayerNormalization(name="layer_norm")
        self.perceptron = tf.keras.layers.Dense(units=perceptron_size,
                                                activation=activation_fn,
                                                kernel_initializer=tf.random_normal_initializer(stddev=initializer_range),
                                                name="perceptron")
        self.projection = tf.keras.layers.Dense(units=embedding_size,
                                                kernel_initializer=tf.random_normal_initializer(stddev=initializer_range),
                                                name="projection")

    def call(self, inputs, dropout=None):
        """

        inputs: tensor of [batch_size, seq_length, dim]

        """
        print(f".... input to    mlp: {tldr(inputs)}")
        x = self.layer_norm(inputs)
        x = self.perceptron(x)
        x = self.projection(x)
        x = dropout_fn(x, dropout)
        print(f".... output from mlp: {tldr(x)}")
        return x

    def __call__(self, inputs, dropout=None):
        return super().__call__(inputs=inputs,
                                dropout=dropout)


class Block(tf.keras.layers.Layer):

    def __init__(self, config, trainable=True, initializer_range=0.02, name=None):
        super().__init__(name=name, trainable=trainable)
        self.attention = AttentionLayer(config=config,
                                        initializer_range=initializer_range,
                                        name="attention")
        self.mlp = MultiLayerPerceptron(activation_fn=gelu,
                                        embedding_size=config["n_embd"],
                                        perceptron_size=4 * config["n_embd"],
                                        initializer_range=initializer_range,
                                        name="mlp")

    def call(self, inputs, cache=None, dropout=None, attention_dropout=None,
            return_cache=False, use_2d=False, shape=None):
        x = inputs
        a = self.attention(inputs=x,
                           cache=cache,
                           dropout=dropout,
                           attention_dropout=attention_dropout,
                           return_cache=return_cache,
                           use_2d=use_2d,
                           shape=shape)
        if return_cache:
            a, cache = a
        x = x + a
        m = self.mlp(inputs=x,
                     dropout=dropout)
        x = x + m
        if return_cache:
            return x, cache
        else:
            return x

    def __call__(self, inputs, cache=None, dropout=None, attention_dropout=None,
                 return_cache=False, use_2d=False, shape=None):
        return super().__call__(inputs=inputs,
                                cache=cache,
                                dropout=dropout,
                                attention_dropout=attention_dropout,
                                return_cache=return_cache,
                                use_2d=use_2d,
                                shape=shape)


class Transformer(tf.keras.Model):

    def __init__(self, config, trainable=True, name=None):
        super().__init__(name=name)
        self.trainable = trainable
        self.blocks = []
        self.blocks_num = config["n_layer"]
        for ids in range(self.blocks_num):
            block = Block(config=config,
                          name="block_%d" % ids)
            self.blocks.append(block)
        self.layer_norm = LayerNormalization(name="layer_norm")

    def call(self, inputs, cache=None, dropout=None, attention_dropout=None,
             return_cache=False, blocks=None, use_2d=False, shape=None):
        """

        inputs: a tensor of shape [batch_size, seq_length, dim], if use_2d is False, else [batch_size * seq_length, dim]
        cache: a list of dictionaries. key and values from previous calls.
        blocks: a list. if it is specified, the output will be a dictionary {layer_num: layer_output}
        return_cache: if it is true, it will returns cache for blocks
        use_2d: if it is True, then the operations will define base on 2D tensors. (for tpu performance)
        shape: if use_2d is True, then it is [batch_size, seq_length]

        """
        if blocks is None:
            max_block = self.blocks_num - 1
        elif len(blocks) == 0:
            max_block = self.blocks_num - 1
            blocks = None
        else:
            _blocks = []
            for i in blocks:
                if i >= 0:
                    k = i
                else:
                    k = self.blocks_num - i
                if k >= self.blocks_num or k < 0:
                    raise ValueError("output blocks should be in range [" + str(0) + ", " +
                                     str(self.blocks_num - 1) + "]")
                _blocks.append(k)
            _blocks = list(sorted(_blocks))
            blocks = _blocks
            max_block = blocks[-1]
        if blocks is not None:
            outputs = {}
        if return_cache:
            new_cache = []
        output = inputs
        for ids in range(max_block + 1):
            if cache is None:
                _cache = None
            else:
                _cache = cache[ids]           
            print()
            print(f".. input to    block {ids}: {tldr(output)}")            
            output = self.blocks[ids](inputs=output,
                                      cache=_cache,
                                      dropout=dropout,
                                      attention_dropout=attention_dropout,
                                      return_cache=return_cache,
                                      use_2d=use_2d,
                                      shape=shape)
            print(f".. output from block {ids}: {tldr(output)}")
            if return_cache:
                output, _cache = output
                new_cache.append(_cache)
            if blocks is not None:
                if ids in blocks:
                    outputs[ids] = output
        if blocks is None:
            output = self.layer_norm(output)
            result = output
        else:
            result = outputs
        if return_cache:
            return result, new_cache
        else:
            return result

    def __call__(self, inputs, cache=None, dropout=None, attention_dropout=None,
                 return_cache=False, blocks=None, use_2d=False, shape=None):
        """

        inputs: a tensor of shape [batch_size, seq_length, dim], if use_2d is False, else [batch_size * seq_length, dim]
        cache: a list of dictionaries. key and values from previous calls.
        blocks: a list. if it is specified, the output will be a dictionary {layer_num: layer_output}
        return_cache: if it is true, it will returns cache for blocks
        use_2d: if it is True, then the operations will define base on 2D tensors. (for tpu performance)
        shape: if use_2d is True, then it is [batch_size, seq_length]

        """
        return super().__call__(
            inputs=inputs,
            cache=cache,
            dropout=dropout,
            attention_dropout=attention_dropout,
            return_cache=return_cache,
            blocks=blocks,
            use_2d=use_2d,
            shape=shape
        )


class Embedding(tf.keras.layers.Layer):

    def __init__(self, embedding_size, vocab_size, max_position_length,
                 trainable=True, name=None, initializer_range=0.02,
                 dtype=None):
        if dtype is None:
            dtype = tf.float32
        super().__init__(name=name, trainable=trainable, dtype=dtype)
        self.word_embedding = None
        self.position_embedding = None
        self.initializer_range = initializer_range
        self.embedding_size = embedding_size
        self.vocab_size = vocab_size
        self.max_position_length = max_position_length

    def build(self, input_shape):
        print("..build")
        self.word_embedding = self.add_weight(
            name="word_embedding",
            shape=(self.vocab_size, self.embedding_size),
            initializer=tf.random_normal_initializer(stddev=self.initializer_range),
        )
        self.position_embedding = self.add_weight(
            name="position_embedding",
            shape=(self.max_position_length, self.embedding_size),
            initializer=tf.random_normal_initializer(stddev=self.initializer_range),
        )

    def call(self, inputs, start=None):
        """

        inputs: integer tensor of [batch_size, seq_length]
        start: start of positional embedding

        """
        shape = get_tensor_shape(inputs)
        x = tf.gather(self.word_embedding, inputs)
        if start is None:
            start = 0
        end = start + shape[1]
        pe = self.position_embedding[start:end]
        x = x + pe
        print(f".. Embedding output: {tldr(x)}")        
        return x

    def __call__(self, inputs, start=None):
        """

        if use_one_hot_keys is True, then inputs are one_hot tensors of shape [batch_size, seq_length, vocab_size],
        else it is an integer tensor of [batch_size, seq_length] of token ids.
        start: start of positional embedding

        """
        return super().__call__(inputs=inputs, start=start)


class GPT2(tf.keras.Model):

    def __init__(self, config, name=None, trainable=True, dtype=None):
        super().__init__(name=name)
        self.trainable = trainable
        self.embedding = Embedding(
            embedding_size=config['n_embd'],
            vocab_size=config['n_vocab'],
            max_position_length=config['n_ctx'],
            name="embedding",
            dtype=dtype
        )
        self.transformer = Transformer(config, name="transformer")

    def call(self, inputs, cache=None,
             dropout=None, attention_dropout=None,
             return_cache=False, return_logits=True, use_2d=False):
        """

        inputs: an integer tensor of shape [batch_size, seq_length] if not use_2d is False
                else a one_hot tensor of shape [batch_size, seq_length, vocab_size]
        cache: a list of dictionaries {"key": key, "value": value} of previous keys and values. it uses for generation
        use_one_hot_keys: if True it uses one hot tensors for embedding layer.
        return_cache: if True returns new keys and values alongside output. it uses for generation.
        return_logits: if True, return logits, else return last layer embedding.
        use_2d: for tpu performances: use 2D tensors for operations and return the output in 2D shape: [batch_size * seq_length, -1]

        """
        if cache is not None:
            _cache = cache[0]["key"]
            start = get_tensor_shape(_cache)[2]
        else:
            start = None
        x = self.embedding(inputs, start)
        if use_2d:
            shape = get_tensor_shape(x)
            x = tf.reshape(x, [shape[0] * shape[1], shape[2]])
            shape = shape[0:2]
        else:
            shape = None
        x = self.transformer(
            inputs=x,
            cache=cache,
            dropout=dropout,
            attention_dropout=attention_dropout,
            return_cache=return_cache,
            use_2d=use_2d,
            shape=shape
        )
        # print("after transformer x.shape=", x.shape)
        # print("after transformer x=", x[0][0][:3].numpy())
        if return_cache:
            x, cache = x
        if return_logits:
            shape = get_tensor_shape(x)
            print("shape:", shape)
            if not use_2d:
                x = tf.reshape(x, [shape[0] * shape[1], shape[2]])
                print("x reshaped:", x.shape)
            logits = tf.matmul(x, self.embedding.word_embedding, transpose_b=True)
            print("logits.shape=", logits.shape)
            if not use_2d:
                logits = tf.reshape(logits, [shape[0], shape[1], self.embedding.vocab_size])
                print("logits reshaped:", tldr(logits))
            result = logits
        else:
            result = x
        if return_cache:
            return result, cache
        else:
            return result

    def __call__(self, inputs, cache=None,
                 dropout=None, attention_dropout=None,
                 return_cache=False, return_logits=True,
                 use_2d=False):
        """

        inputs: an integer tensor of shape [batch_size, seq_length]
        cache: a list of dictionaries {"key": key, "value": value} of previous keys and values. it uses for generation
        use_one_hot_keys: if True it uses one hot tensors for embedding layer.
        return_cache: if True returns new keys and values alongside output. it uses for generation.
        return_logits: if True, return logits, else return last layer embedding.
        use_2d: for tpu performances: use 2D tensors for operations and return the output in 2D shape: [batch_size * seq_length, -1]

        """
        return super().__call__(
            inputs=inputs,
            cache=cache,
            dropout=dropout,
            attention_dropout=attention_dropout,
            return_cache=return_cache,
            return_logits=return_logits,
            use_2d=use_2d
        )



### Test Embedding in isolation

In [36]:
embedding_layer = Embedding(embedding_size=768, vocab_size=50257, max_position_length=1024)
_ = embedding_layer(tf.constant([[1]]))
assert embedding_layer.built
# embedding_layer.build(input_shape=(None,768))
# embedding_layer.built
# embedding_layer.word_embedding     = copy.deepcopy(params['wte']) # word_embedding: (50257, 768) self.vocab_size, self.embedding_size
# embedding_layer.position_embedding = copy.deepcopy(params['wpe']) # position_embedding: (1024, 768) max_position_length, self.embedding_size


..build
.. Embedding output: float32 (1, 1, 768) [-0.01077  0.0535  -0.03079]


In [37]:
embedding_layer.built

True

In [38]:
embedding_layer.word_embedding     = copy.deepcopy(params['wte']) # word_embedding: (50257, 768) self.vocab_size, self.embedding_size
embedding_layer.position_embedding = copy.deepcopy(params['wpe']) # position_embedding: (1024, 768) max_position_length, self.embedding_size

In [39]:
assert embedding_layer.word_embedding.shape == (50257, 768)
assert np.allclose(embedding_layer.word_embedding[0][:3], np.array([ -0.1101, -0.03927,  0.03311]), rtol=1e-3, atol=1e-3)

In [40]:
x_effort = tf.constant([[6109, 3626, 6100, 345]])
x_effort_emb = embedding_layer(x_effort)
assert x_effort_emb.shape == [1, 4, 768]
assert np.allclose(x_effort_emb[0][0][:3], np.array([0.0793, -0.2979 ,  0.0882]), rtol=1e-4, atol=1e-4)

.. Embedding output: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]


In [41]:
config124M = {'n_embd': 768, 'n_vocab': 50257, 'n_ctx': 1024, 'n_layer': 12, 'n_head': 12}
# # config = {'n_embd': 3, 'n_vocab': 10, 'n_ctx': 5, 'n_layer': 12, 'n_head': 4}
gpt2 = GPT2(name="mygpt2", config=config124M, trainable=False)
# x=tf.constant([[1]])
# gpt2.compile()

In [44]:
# gpt2.summary(expand_nested=True)
# print_layer_structure(gpt2)

In [45]:
# checkpoint = tf.train.Checkpoint(gpt2)
# model_dir = 'tf_ckpts'
# save_path = checkpoint.save(model_dir + "/ckpt")
# # list all the variables in the model
# tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
# for name, v in tf.train.list_variables(tf_ckpt_path):
#     # print(name)
#     pass

## Load weights into params dict

In [46]:
settings = {"n_layer": 12}

model_dir="ch05/01_main-chapter-code/gpt2/124M"
tf_ckpt_path = tf.train.latest_checkpoint(model_dir)
params = load_gpt2_params_from_tf_ckpt(tf_ckpt_path, settings)

In [47]:
# Let's study this "params" thing..
params.keys() # ['blocks', 'b', 'g', 'wpe', 'wte']
len(params["blocks"]) # 12
params_block0 = params["blocks"][0]
params_block0.keys() # ['attn', 'ln_1', 'ln_2', 'mlp']
params_block0_attn = params_block0['attn']
params_block0_attn.keys() # ['c_attn', 'c_proj']
params_block0_attn_c_attn = params_block0_attn['c_attn']
params_block0_attn_c_attn.keys() # ['b', 'w']
x = (params["blocks"][0]["attn"]["c_attn"])["w"]
x.shape # (768, 2304)
q_w, k_w, v_w = np.split((params["blocks"][0]["attn"]["c_attn"])["w"], 3, axis=-1)
q_b, k_b, v_b = np.split((params["blocks"][0]["attn"]["c_attn"])["b"], 3, axis=-1)
print("q_w.shape:", q_w.shape) # (768, 768)
print("q_b.shape:", q_b.shape) # (768,)

q_w.shape: (768, 768)
q_b.shape: (768,)


In [48]:
# params['wpe'].shape # (1024, 768) position embedding
params['wte'].shape # (50257, 768) token embedding, out_head.weight
#len(params['blocks']) 12
#params['blocks'][0].keys() # dict_keys(['attn', 'ln_1', 'ln_2', 'mlp'])
# params['b'].shape # (768,) final_norm.shift (beta)
# params['g'].shape # (768,) final_norm.scale (gamma)

(50257, 768)

In [49]:
b = 0
params["blocks"][b]["ln_1"]["b"].shape # (768,) beta
params["blocks"][b]["ln_1"]["g"].shape  # (768,) gamma

(768,)

In [50]:
gpt2.get_config()

{'name': 'mygpt2',
 'trainable': False,
 'dtype': 'float32',
 'config': {'n_embd': 768,
  'n_vocab': 50257,
  'n_ctx': 1024,
  'n_layer': 12,
  'n_head': 12}}

## Here we load all the weights!!

In [51]:
# a GPT Model has an Embedding layer and a Transformer Model
# embedding_layer   = gpt2.embedding

# embedding_layer = Embedding(embedding_size=768, vocab_size=50257, max_position_length=1024)
_ = gpt2.embedding(tf.constant([[1]]))
assert gpt2.embedding.built
# The Embedding Layer has word_embedding, position_embedding, initializer_range, embedding_size, vocab_size, max_position_length
gpt2.embedding.word_embedding     = copy.deepcopy(params['wte']) # word_embedding: (50257, 768) self.vocab_size, self.embedding_size
gpt2.embedding.position_embedding = copy.deepcopy(params['wpe']) # position_embedding: (1024, 768) max_position_length, self.embedding_size
assert gpt2.embedding.word_embedding.shape == (50257, 768)
assert np.allclose(gpt2.embedding.word_embedding[0][:3], np.array([ -0.1101, -0.03927,  0.03311]), rtol=1e-3, atol=1e-3)

..build
.. Embedding output: float32 (1, 1, 768) [ 0.06137 -0.02885  0.03488]


In [52]:
x_768ones = np.ones((1, 768) , dtype=np.float32)
_ = gpt2.transformer.blocks[0].attention.layer_norm(tf.constant(x_768ones))


In [53]:
gpt2.transformer.blocks[0].attention.layer_norm.built

True

In [54]:
# transformer_layer = gpt2.transformer
x_768ones = np.ones((1, 768) , dtype=np.float32)
blocks = []
for b in range(gpt2.get_config()['config']['n_layer']): # = transformer_layer.blocks_num (12)
  # A transformer_layer has a list of blocks
  print(f"..loading block {b}")
  block = gpt2.transformer.blocks[b]
  blocks.append(block)

  _ = gpt2.transformer.blocks[b].attention.layer_norm(tf.constant(x_768ones))
  assert gpt2.transformer.blocks[0].attention.layer_norm.built
  gpt2.transformer.blocks[b].attention.layer_norm.beta = copy.deepcopy(params["blocks"][b]["ln_1"]["b"]) # Not real sure about these..
  gpt2.transformer.blocks[b].attention.layer_norm.gamma = copy.deepcopy(params["blocks"][b]["ln_1"]["g"])

  gpt2.transformer.blocks[b].attention.self_attention.query_layer.build((None, 768))
  assert gpt2.transformer.blocks[b].attention.self_attention.query_layer.built
  gpt2.transformer.blocks[b].attention.self_attention.key_layer.build((None, 768))
  assert gpt2.transformer.blocks[b].attention.self_attention.key_layer.built
  gpt2.transformer.blocks[b].attention.self_attention.value_layer.build((None, 768))
  assert gpt2.transformer.blocks[b].attention.self_attention.value_layer.built
  q_w, k_w, v_w = np.split((params["blocks"][b]["attn"]["c_attn"])["w"], 3, axis=-1)
  q_b, k_b, v_b = np.split((params["blocks"][b]["attn"]["c_attn"])["b"], 3, axis=-1)
  gpt2.transformer.blocks[b].attention.self_attention.query_layer.set_weights([q_w, q_b])
  gpt2.transformer.blocks[b].attention.self_attention.key_layer.set_weights([k_w, k_b])
  gpt2.transformer.blocks[b].attention.self_attention.value_layer.set_weights([v_w, v_b])

  gpt2.transformer.blocks[b].attention.projection.build((None, 768))
  assert gpt2.transformer.blocks[b].attention.projection.built
  gpt2.transformer.blocks[b].attention.projection.set_weights([params["blocks"][b]["attn"]["c_proj"]["w"], params["blocks"][b]["attn"]["c_proj"]["b"]])

  _ = gpt2.transformer.blocks[b].mlp.layer_norm(tf.constant(x_768ones))
  assert gpt2.transformer.blocks[b].mlp.layer_norm.built
  gpt2.transformer.blocks[b].mlp.layer_norm.beta = params["blocks"][b]["ln_2"]["b"] # Not real sure about these..
  gpt2.transformer.blocks[b].mlp.layer_norm.gamma = params["blocks"][b]["ln_2"]["g"]

    
  gpt2.transformer.blocks[b].mlp.perceptron.build((None, 768))
  assert gpt2.transformer.blocks[b].mlp.perceptron.built
  gpt2.transformer.blocks[b].mlp.perceptron.set_weights([params["blocks"][b]["mlp"]["c_fc"]["w"], params["blocks"][b]["mlp"]["c_fc"]["b"]])
  gpt2.transformer.blocks[b].mlp.projection.build((None, 3072))
  assert gpt2.transformer.blocks[b].mlp.projection.built
  gpt2.transformer.blocks[b].mlp.projection.set_weights([params["blocks"][b]["mlp"]["c_proj"]["w"], params["blocks"][b]["mlp"]["c_proj"]["b"]])

gpt2.transformer.layer_norm(tf.constant(x_768ones))
assert gpt2.transformer.layer_norm.built
gpt2.transformer.layer_norm.beta = copy.deepcopy(params["b"])
gpt2.transformer.layer_norm.gamma = copy.deepcopy(params["g"])

..loading block 0
..loading block 1
..loading block 2
..loading block 3
..loading block 4
..loading block 5
..loading block 6
..loading block 7
..loading block 8
..loading block 9
..loading block 10
..loading block 11


## Test each layer, comparing to corresponding LLMFS Pytorch layer 

### test embedding_layer

In [10]:
assert np.allclose(gpt2.embedding.word_embedding[0][:3], np.array([ -0.1101, -0.03927,  0.03311]), rtol=1e-3, atol=1e-3)

x_123 = tf.constant([[1, 2, 3]])
x = gpt2.embedding(x_123)
assert x.shape == [1, 3, 768]
assert np.allclose(x[0][0][:3], np.array([ 0.02151961, -0.24603364,  0.05027542]), rtol=1e-3, atol=1e-3)

x_effort = tf.constant([[6109, 3626, 6100, 345]])
x_effort_emb = gpt2.embedding(x_effort)
assert x_effort_emb.shape == [1, 4, 768]
assert np.allclose(x_effort_emb[0][0][:3], np.array([0.0793, -0.2979 ,  0.0882]), rtol=1e-4, atol=1e-4)

.. Embedding output: float32 (1, 3, 768) [ 0.02152 -0.24603  0.05028]
.. Embedding output: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]


In [11]:
assert np.allclose(gpt2.embedding.word_embedding[0][:3], np.array([ -0.1101, -0.03927,  0.03311]), rtol=1e-3, atol=1e-3)

### test layer_norm

In [12]:
b=0
assert np.allclose(gpt2.transformer.blocks[b].attention.layer_norm.beta[:3], np.array([-3.67733e-03,  2.71967e-02, -6.40409e-02 ]), rtol=1e-4, atol=1e-4)


In [13]:
# gpt2.transformer.blocks[b].attention.layer_norm.beta

In [14]:
# test layer_norm
x_768ones = np.ones((1, 768) , dtype=np.float32)

layer_norm0 = blocks[0].attention.layer_norm
# layer_norm0(x) # [-3.6773e-03,  2.7197e-02, -6.4041e-02
# layer_norm11 = blocks[11].attention.layer_norm
# layer_norm11(x) # [ 5.0957e-02,  5.3063e-03,  7.1952e-02
x = layer_norm0(x_768ones)
assert x.shape == [1, 768]
assert np.allclose(x[0][:3], np.array([-3.67732509e-03,  2.71967370e-02, -6.40409067e-02]), rtol=1e-3, atol=1e-3)

In [15]:
x = gpt2.transformer.blocks[0].attention.layer_norm(x_768ones)
assert x.shape == [1, 768]
assert np.allclose(x[0][:3], np.array([-3.67732509e-03,  2.71967370e-02, -6.40409067e-02]), rtol=1e-3, atol=1e-3)

In [16]:
# test query_layer, key_layer, value_layer, layer_proj
x = tf.constant(np.ones((1, 768) , dtype=np.float32))
# block 0
query_layer0 = blocks[0].attention.self_attention.query_layer
#query_layer0(x) # [-1.3708e+01,  1.3385e+01,  1.4323e+01
key_layer0 = blocks[0].attention.self_attention.key_layer
# key_layer0(x) [ 1.8049e-01, -1.4381e-01,  6.2964e-01
value_layer0 = blocks[0].attention.self_attention.value_layer               
# value_layer0(x) # [-6.1687e-02, -1.3786e-01, -3.0145e-01
layer_proj0 = blocks[0].attention.projection
# layer_proj0(x) # [-9.7561e+00, -1.7296e+01, -6.7800e-01
# layer_proj0(value_layer0(key_layer0(query_layer0(x)))) # [-2.3273e+01, -7.9272e+02,  5.6245e+02

# block 11
query_layer11 = blocks[11].attention.self_attention.query_layer
# query_layer11(x) # [-5.4209e+00,  4.6236e+00,  4.5401e+00
key_layer11 = blocks[11].attention.self_attention.key_layer
# key_layer11(x) # [ 5.8911e+00, -3.3184e-01,  6.3656e-01
value_layer11 = blocks[11].attention.self_attention.value_layer               
# value_layer11(x) # [-1.2480e+00, -3.0783e+00,  5.9679e+00
layer_proj11 = blocks[11].attention.projection
#layer_proj11(x) # [-4.1535e-01,  2.1763e+00,  4.7958e-01

# layer_proj11(value_layer11(key_layer11(query_layer11(x)))) # [ 3.4414e+02,  4.9568e+02,  3.8639e+02



## Validate each layer to compare with LLMFS and gpt2s

### Test embedding

In [17]:
x_effort = tf.constant([[6109, 3626, 6100, 345]])

x_effort_emb = gpt2.embedding(x_effort)
assert x_effort_emb.shape == [1, 4, 768]
assert x_effort_emb.shape == [1, 4, 768]
assert np.allclose(x_effort_emb[0][0][:3], np.array([0.0793, -0.2979 ,  0.0882]), rtol=1e-4, atol=1e-4)

.. Embedding output: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]


#### Test AttentionLayer layer_norm

In [31]:
b=0

y = gpt2.transformer.blocks[b].attention.layer_norm(x_effort_emb)
assert y.shape == [1, 4, 768]
assert np.allclose(y[0][0][:3], np.array([0.047223  , -0.11664161, -0.02536647]), rtol=1e-3, atol=1e-3)

#### Test AttentionLayer SelfAttention

In [21]:
b = 0
self_attention = gpt2.transformer.blocks[b].attention.self_attention
y = self_attention(x_effort_emb)
assert y.shape == [1, 4, 768]
assert np.allclose(y[0][0][:3], np.array([0.28434375, -0.00881347,  0.34210888]), rtol=1e-3, atol=1e-3)

.... input to    SelfAttention: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]
.... output from SelfAttention: float32 (1, 4, 768) [ 0.28434 -0.00881  0.34211]


#### Test AttentionLayer projection

In [22]:
b=0
# gpt2s.transformer.blocks[b].attention.projection.build((None, 768))
# gpt2s.transformer.blocks[b].attention.projection.set_weights([params["blocks"][b]["attn"]["c_proj"]["w"], params["blocks"][b]["attn"]["c_proj"]["b"]])

y = gpt2.transformer.blocks[b].attention.projection(x_effort_emb)
assert y.shape == [1, 4, 768]
assert np.allclose(y[0][0][:3], np.array([2.5602593 ,  0.34704542,  0.3729586]), rtol=1e-3, atol=1e-3)


#### Test the entire AttentionLayer

In [23]:
y = gpt2.transformer.blocks[b].attention.projection(self_attention(gpt2.transformer.blocks[b].attention.layer_norm(x_effort_emb)))
assert y.shape == [1, 4, 768]
assert np.allclose(y[0][0][:3], np.array([5.4214954e-01, -1.1554953e-01,  2.5736535e-01]), rtol=1e-3, atol=1e-3)

.... input to    SelfAttention: float32 (1, 4, 768) [ 0.04722 -0.11664 -0.02537]
.... output from SelfAttention: float32 (1, 4, 768) [ 0.09438  0.02312 -0.01932]


In [24]:
b=0
y = gpt2.transformer.blocks[b].attention(x_effort_emb)
assert y.shape == [1, 4, 768]
assert np.allclose(y[0][0][:3], np.array([5.4214954e-01, -1.1554953e-01,  2.5736535e-01]), rtol=1e-3, atol=1e-3)


.... input to    layer_norm: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]
.... output from layer_norm: float32 (1, 4, 768) [ 0.04722 -0.11664 -0.02537]
.... input to    SelfAttention: float32 (1, 4, 768) [ 0.04722 -0.11664 -0.02537]
.... output from SelfAttention: float32 (1, 4, 768) [ 0.09438  0.02312 -0.01932]
.... input to    projection: float32 (1, 4, 768) [ 0.09438  0.02312 -0.01932]
.... output from projection: float32 (1, 4, 768) [ 0.54215 -0.11555  0.25737]


#### Test MultiLayerPerceptron layer_norm

In [25]:
y = gpt2.transformer.blocks[b].mlp.layer_norm(x_effort_emb)
assert y.shape == [1, 4, 768]
assert np.allclose(y[0][0][:3], np.array([0.0723421 , -0.1328541 ,  0.0565621]), rtol=1e-3, atol=1e-3)


#### Test MultiLayerPerceptron perceptron

In [26]:
b=0
# x_effort_emb_3072 = tf.concat([tf.concat([x_effort_emb, x_effort_emb], axis=-1), tf.concat([x_effort_emb, x_effort_emb], axis=-1)], axis=-1)
y = gpt2.transformer.blocks[b].mlp.perceptron(x_effort_emb)
assert y.shape == [1, 4, 3072]
assert np.allclose(y[0][0][:3], np.array([-1.26867130e-01, -5.54500334e-02, -1.59711763e-02]), rtol=1e-3, atol=1e-3)
#y

#### Test MultiLayerPerceptron projection

In [27]:
b=0

x_effort_emb_3072 = tf.concat([tf.concat([x_effort_emb, x_effort_emb], axis=-1), tf.concat([x_effort_emb, x_effort_emb], axis=-1)], axis=-1)
y = gpt2.transformer.blocks[b].mlp.projection(x_effort_emb_3072)
assert y.shape == [1, 4, 768]
y
assert np.allclose(y[0][0][:3], np.array([-0.8513732 ,  1.9853125 , -2.6772308]), rtol=1e-3, atol=1e-3)

#### Test the entire layer MultiLayerPerceptron

In [28]:
b = 0
y = gpt2.transformer.blocks[b].mlp(x_effort_emb)
assert y.shape == [1, 4, 768]
assert np.allclose(y[0][0][:3], np.array([3.2065992,   2.262373 ,   1.4517794]), rtol=1e-3, atol=1e-3)

.... input to    mlp: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]
.... output from mlp: float32 (1, 4, 768) [3.2066  2.26237 1.45178]


#### Test Transformer layer_norm

In [29]:

y = gpt2.transformer.layer_norm(x_effort_emb)
assert y.shape == [1, 4, 768]
assert np.allclose(y[0][0][:3], np.array([0.3196596 , -1.050371  ,  0.4083333]), rtol=1e-3, atol=1e-3)


### Test the whole model!!

In [55]:
y = gpt2(x_effort)

.. Embedding output: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]

.. input to    block 0: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]
.... input to    layer_norm: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]
.... output from layer_norm: float32 (1, 4, 768) [ 0.04722 -0.11664 -0.02537]
.... input to    SelfAttention: float32 (1, 4, 768) [ 0.04722 -0.11664 -0.02537]
.... output from SelfAttention: float32 (1, 4, 768) [ 0.09438  0.02312 -0.01932]
.... input to    projection: float32 (1, 4, 768) [ 0.09438  0.02312 -0.01932]
.... output from projection: float32 (1, 4, 768) [ 0.54215 -0.11555  0.25737]
.... input to    mlp: float32 (1, 4, 768) [ 0.62142 -0.41347  0.34554]
.... output from mlp: float32 (1, 4, 768) [0.76306 0.73464 0.57444]
.. output from block 0: float32 (1, 4, 768) [1.38448 0.32117 0.91998]

.. input to    block 1: float32 (1, 4, 768) [1.38448 0.32117 0.91998]
.... input to    layer_norm: float32 (1, 4, 768) [1.38448 0.32117 0.91998]
.... output from layer_norm

In [56]:
tldr(y)

'float32 (1, 4, 50257) [-35.58201 -34.98038 -38.45218]'

In [33]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#

## Here we are validating SelfAttention

In [33]:
layer_norm0 = blocks[0].attention.layer_norm

# x = layer_norm0(x_123)

In [35]:
x_123

<tf.Tensor: shape=(1, 3), dtype=int32, numpy=array([[1, 2, 3]], dtype=int32)>

In [34]:
b = 0
self_attention = gpt2.transformer.blocks[b].attention.self_attention

x = gpt2.embedding(x_123)

self_attention(x)

SelfAttention: inputs.shape= (1, 3, 768) tf.Tensor([ 0.02151961 -0.24603364  0.05027542], shape=(3,), dtype=float32)
SelfAttention: query.shape= (1, 3, 768) tf.Tensor([-1.4599288  2.6167424  3.0795689], shape=(3,), dtype=float32)
SelfAttention: reshape query.shape= (1, 12, 3, 64) tf.Tensor([-1.4599288  2.6167424  3.0795689], shape=(3,), dtype=float32)
SelfAttention: reshape coefficients.shape= (1, 12, 3, 3) tf.Tensor([37.56014  20.718346 18.429935], shape=(3,), dtype=float32)
SelfAttention: results coefficients.shape= (1, 12, 3, 64) tf.Tensor([0.37184218 0.04303181 0.43206325], shape=(3,), dtype=float32)


<tf.Tensor: shape=(1, 3, 768), dtype=float32, numpy=
array([[[ 0.37184218,  0.04303181,  0.43206325, ...,  0.21111687,
         -0.346839  ,  1.209221  ],
        [ 0.371842  ,  0.0430319 ,  0.4320631 , ...,  0.4688711 ,
          0.6618678 ,  0.8360132 ],
        [ 0.37184218,  0.04303183,  0.43206328, ...,  0.61793643,
          0.34492874,  0.9636562 ]]], dtype=float32)>

In [22]:
x = gpt2.embedding(x_123)
x = gpt2.transformer.blocks[b].attention.projection(x)
x

<tf.Tensor: shape=(1, 3, 768), dtype=float32, numpy=
array([[[ 2.9546063 ,  1.2646755 , -0.8381394 , ..., -0.03054229,
         -0.21869148,  0.3622729 ],
        [ 0.1328235 , -0.6140044 ,  1.6673647 , ...,  0.14359668,
         -0.05004777, -0.01445162],
        [-1.7971708 , -0.2732838 ,  1.5349535 , ...,  0.06378165,
         -0.03412376,  0.03880764]]], dtype=float32)>

### Here we are validating layer_norm in the AttentionLayer

In [39]:
x = gpt2.embedding(x_123)
gpt2.transformer.blocks[b].attention.layer_norm(x)


<tf.Tensor: shape=(1, 3, 768), dtype=float32, numpy=
array([[[ 0.00984592, -0.09290446, -0.04286208, ...,  0.01140726,
          0.00143049,  0.03473373],
        [-0.10779594,  0.02115111, -0.00442008, ...,  0.09210561,
         -0.11231511, -0.09120307],
        [-0.09531845, -0.30332777,  0.12632102, ...,  0.16344029,
          0.03355676, -0.19331908]]], dtype=float32)>

#### test the entire Attention layer

In [36]:
attention = gpt2.transformer.blocks[b].attention
x = gpt2.embedding(x_123)

attention(x)

SelfAttention: inputs.shape= (1, 3, 768) tf.Tensor([ 0.00984592 -0.09290446 -0.04286208], shape=(3,), dtype=float32)
SelfAttention: query.shape= (1, 3, 768) tf.Tensor([-0.04680172 -0.4413777   0.29562825], shape=(3,), dtype=float32)
SelfAttention: reshape query.shape= (1, 12, 3, 64) tf.Tensor([-0.04680172 -0.4413777   0.29562825], shape=(3,), dtype=float32)
SelfAttention: reshape coefficients.shape= (1, 12, 3, 3) tf.Tensor([-0.6131798 -1.2038919 -1.8434243], shape=(3,), dtype=float32)
SelfAttention: results coefficients.shape= (1, 12, 3, 64) tf.Tensor([0.14607082 0.05066063 0.02101883], shape=(3,), dtype=float32)


<tf.Tensor: shape=(1, 3, 768), dtype=float32, numpy=
array([[[ 3.3282498e-01,  8.7829292e-02,  1.8114735e-01, ...,
         -3.9580278e-02, -8.0668032e-03,  3.5296157e-02],
        [ 6.3773024e-01, -8.8449240e-02, -1.6967435e-01, ...,
         -4.4448502e-02,  5.2359477e-03,  5.8977678e-04],
        [-4.9767217e-01,  1.8594694e-01,  1.2540229e-01, ...,
         -2.5778674e-02, -7.1540968e-03,  4.3813903e-03]]], dtype=float32)>

In [19]:
# test mlp_layer_norm, mlp_perceptron, mlp_projection
x = np.ones((1, 768) , dtype=np.float32)
mlp_layer_norm0 = blocks[0].mlp.layer_norm
# mlp_layer_norm0(x) # [ 4.2478e-02,  3.2627e-02,  4.4881e-03

mlp_perceptron0 = blocks[0].mlp.perceptron
mlp_projection0 = blocks[0].mlp.projection
# mlp_projection0(mlp_perceptron0(x)) # [-1.6735e+01, -6.9883e+00,  4.1138e+00

mlp_layer_norm11 = blocks[11].mlp.layer_norm
# mlp_layer_norm11(x) # [-1.9770e-03,  2.0055e-02,  3.8334e-02

mlp_perceptron11 = blocks[11].mlp.perceptron
mlp_projection11 = blocks[11].mlp.projection
#mlp_projection11(mlp_perceptron11(x)) # [ 1.3675e+01,  2.2839e+01, -1.7306e+01


In [20]:
x_trivial = tf.constant([[1, 2, 3]])
gpt2(x_trivial)

1 x: [ 0.02151961 -0.24603364  0.05027542]
2 x: [-0.05177143  0.12072419 -0.42840737]


<tf.Tensor: shape=(1, 3, 50257), dtype=float32, numpy=
array([[[-32.90102 , -31.202356, -34.662197, ..., -39.48669 ,
         -39.873096, -32.23865 ],
        [-55.520763, -53.42853 , -56.476704, ..., -68.153885,
         -66.77085 , -58.60061 ],
        [-61.79686 , -60.538612, -59.550335, ..., -75.32061 ,
         -72.773125, -65.57064 ]]], dtype=float32)>

In [21]:
# word_emb = gpt2.word_emb
# pos_emb = gpt2.pos_emb
# we = word_emb(x_trivial) # .shape # TensorShape([1, 3, 768])
# pe = pos_emb(tf.range(1024)) # .shape # TensorShape([1024, 768])
# x = we + pe
emb = gpt2.embedding
emb(x_trivial)

<tf.Tensor: shape=(1, 3, 768), dtype=float32, numpy=
array([[[ 0.02151961, -0.24603364,  0.05027542, ...,  0.04301079,
          0.03080702,  0.09767969],
        [-0.10350236, -0.00585408,  0.0892228 , ...,  0.12408535,
         -0.11955193, -0.08801492],
        [-0.08849797, -0.39009592,  0.26571876, ...,  0.19665493,
          0.055693  , -0.2011495 ]]], dtype=float32)>

In [30]:
def generate_text_simple(model, idx, max_new_tokens, context_size):
    idx = tf.cast(idx, dtype=tf.int64)
    for i in range(max_new_tokens):
        idx_cond = idx
        # print("i=", i, "idx_cond=", idx_cond)
        logits = model(idx)
        logits = logits[:, -1, :]
        probas = tf.nn.softmax(logits, -1)
        idx_next = tf.argmax(logits)
        idx_next = tf.argmax(probas, -1)        
        # print("idx_next:", idx_next)
        idx_next_expanded = tf.expand_dims(idx_next, axis=0)
        # print("idx_next_expanded:", idx_next_expanded)
        idx = tf.concat((idx, idx_next_expanded), axis=1)
        # print("idx_next_expanded after concat:", idx_next_expanded)
        # print("idx               after concat:", idx)
    return idx

In [31]:
idx = tf.constant([[6109 , 3626, 6100,  345]])
token_ids = generate_text_simple(gpt2, idx=idx, max_new_tokens=5, context_size=256)
print("token_ids:", token_ids) # [6109, 3626, 6100,  345, 2651,   13,  198,  198,  464]

.. Embedding output: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]

.. input to    block 0: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]
.... input to    layer_norm: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]
.... output from layer_norm: float32 (1, 4, 768) [ 0.04722 -0.11664 -0.02537]
.... input to    SelfAttention: float32 (1, 4, 768) [ 0.04722 -0.11664 -0.02537]
.... output from SelfAttention: float32 (1, 4, 768) [ 0.09438  0.02312 -0.01932]
.... input to    projection: float32 (1, 4, 768) [ 0.09438  0.02312 -0.01932]
.... output from projection: float32 (1, 4, 768) [ 0.54215 -0.11555  0.25737]
.... input to    mlp: float32 (1, 4, 768) [ 0.62142 -0.41347  0.34554]
.... output from mlp: float32 (1, 4, 768) [0.76306 0.73464 0.57444]
.. output from block 0: float32 (1, 4, 768) [1.38448 0.32117 0.91998]

.. input to    block 1: float32 (1, 4, 768) [1.38448 0.32117 0.91998]
.... input to    layer_norm: float32 (1, 4, 768) [1.38448 0.32117 0.91998]
.... output from layer_norm

In [24]:
idx = tf.constant([[6109 , 3626, 6100,  345]])
print(idx)
# print(idx[:, -1, :])
# generate(gpt2, idx=idx, max_new_tokens=3, context_size=256)
logits = gpt2(idx)
print("logits:\n", logits)
logits = logits[:, -1, :]
print("logits:\n", logits)
print("logits[-1]:\n", logits[-1])
probas = tf.nn.softmax(logits, -1)
print("probas:\n", probas)
idx_next = tf.argmax(probas, -1)
print("idx_next:\n", idx_next)

tf.Tensor([[6109 3626 6100  345]], shape=(1, 4), dtype=int32)
1 x: [ 0.07927368 -0.2979193   0.08817437]
2 x: [-0.00448901  0.08728749 -0.2910273 ]
logits:
 tf.Tensor(
[[[ -35.58201   -34.980377  -38.45218  ...  -42.09592   -41.85323
    -35.59654 ]
  [ -76.96015   -76.69698   -81.930885 ...  -88.798386  -86.76316
    -78.962685]
  [-125.34872  -126.2704   -135.09477  ... -132.31728  -135.25441
   -127.65115 ]
  [-136.6002   -137.38036  -146.5556   ... -148.2978   -147.21553
   -139.56772 ]]], shape=(1, 4, 50257), dtype=float32)
logits:
 tf.Tensor([[-136.6002  -137.38036 -146.5556  ... -148.2978  -147.21553 -139.56772]], shape=(1, 50257), dtype=float32)
logits[-1]:
 tf.Tensor([-136.6002  -137.38036 -146.5556  ... -148.2978  -147.21553 -139.56772], shape=(50257,), dtype=float32)
probas:
 tf.Tensor(
[[1.6012554e-03 7.3391397e-04 7.6012654e-08 ... 1.3312417e-08
  3.9290100e-08 8.2354178e-05]], shape=(1, 50257), dtype=float32)
idx_next:
 tf.Tensor([2651], shape=(1,), dtype=int64)


In [32]:

tokenizer = tiktoken.get_encoding("gpt2")
def text_to_token_ids(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    # encoded_tensor = torch.tensor(encoded).unsqueeze(0) # add batch dimension
    encoded_tensor = tf.constant(encoded) #.unsqueeze(0) # add batch dimension
    encoded_tensor = tf.expand_dims(encoded_tensor, axis=0)
    return encoded_tensor

def token_ids_to_text(token_ids, tokenizer):
    # flat = token_ids.squeeze(0) # remove batch dimension
    # return tokenizer.decode(flat.tolist())
    return tokenizer.decode(token_ids[-1])

token_ids = text_to_token_ids("Every effort moves you", tokenizer)
print("token_ids:", token_ids)
print(token_ids_to_text(token_ids, tokenizer))

token_ids: tf.Tensor([[6109 3626 6100  345]], shape=(1, 4), dtype=int32)
Every effort moves you


In [33]:
start_context = "Every effort moves you"
token_ids = generate_text_simple(
    model=gpt2,
    idx=text_to_token_ids(start_context, tokenizer),
    max_new_tokens=10,
    context_size=256
)

print("Output text:\n", token_ids_to_text(token_ids, tokenizer))

.. Embedding output: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]

.. input to    block 0: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]
.... input to    layer_norm: float32 (1, 4, 768) [ 0.07927 -0.29792  0.08817]
.... output from layer_norm: float32 (1, 4, 768) [ 0.04722 -0.11664 -0.02537]
.... input to    SelfAttention: float32 (1, 4, 768) [ 0.04722 -0.11664 -0.02537]
.... output from SelfAttention: float32 (1, 4, 768) [ 0.09438  0.02312 -0.01932]
.... input to    projection: float32 (1, 4, 768) [ 0.09438  0.02312 -0.01932]
.... output from projection: float32 (1, 4, 768) [ 0.54215 -0.11555  0.25737]
.... input to    mlp: float32 (1, 4, 768) [ 0.62142 -0.41347  0.34554]
.... output from mlp: float32 (1, 4, 768) [0.76306 0.73464 0.57444]
.. output from block 0: float32 (1, 4, 768) [1.38448 0.32117 0.91998]

.. input to    block 1: float32 (1, 4, 768) [1.38448 0.32117 0.91998]
.... input to    layer_norm: float32 (1, 4, 768) [1.38448 0.32117 0.91998]
.... output from layer_norm